* Purpose: Script to extract holding/location information from worldcat api results   
* License: GPLv3 (Free Software) 
* Date: Oct 15, 2018
* Aditional Notes: API Get Request: #http://www.worldcat.org/webservices/catalog/content/libraries/62123162?wskey={built-in-api-key}

In [7]:
# Load required libraries
from urllib.request import urlopen
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
import xml.etree.ElementTree as ET
from geopy.geocoders import Nominatim

In [8]:
def get_holdings(oclc_number, startLibrary):
    oclc_url = 'http://www.worldcat.org/webservices/catalog/content/libraries/' + oclc_number + '?location=m1b6j4&wskey={built-in-api-key}&startLibrary='+ str(startLibrary) + '&maximumLibraries=100'
    print(oclc_url)
    hodings_info = urlopen(oclc_url).read()
    page_xml = BeautifulSoup(hodings_info, "xml")
    diagnostics = page_xml.findAll("diagnostics")
    
    holdings = None
    records_status = False
    if len(diagnostics) > 0:
        records_status = False
    else:
        holdings = page_xml.findAll("holdings")[0]
        records_status = True
    
    return holdings, records_status

In [9]:
def add_holdings_records_to_df(oclc_number, holdings_xml, location_info_df):
    table_results = holdings_xml.findAll("holding")
    holding_page_count = len(table_results)
    
    for holding in table_results:
        institutionIdentifier = holding.find("institutionIdentifier").find("value").text.strip()
        physicalLocation = holding.find("physicalLocation").text.strip()
        physicalAddress = holding.find("physicalAddress").text.strip()
        copiesCount = holding.find("copiesCount").text.strip()
        location_info_df = location_info_df.append({'oclc_number':oclc_number, 'institutionIdentifier': institutionIdentifier, 'physicalLocation': physicalLocation, 'physicalAddress': physicalAddress, 'copiesCount': copiesCount}, ignore_index=True)    
    return location_info_df, holding_page_count

In [10]:
def get_all_holdings_records(oclc_number):
    has_more_records = True
    startLibrary = 1
    location_info_df = pd.DataFrame()

    while has_more_records == True:
        holdings_xml, records_status = get_holdings(oclc_number, startLibrary)
        if records_status == False:
            break
        location_info_df, holding_page_count = add_holdings_records_to_df(oclc_number, holdings_xml, location_info_df)
        if holding_page_count == 100:
            startLibrary = startLibrary + 100
        else:
            has_more_records = False
    location_info_df.to_csv("data/extracted/holdings/hodings_info_" + oclc_number + ".csv", header=True, columns=["oclc_number","institutionIdentifier","physicalLocation","physicalAddress","copiesCount","latitude","longitude"])            

In [11]:
oclc_works = pd.read_csv("data/source/sociology_oclc_info.csv")
oclc_works.head()

for index, row in oclc_works.iterrows():
    oclc_number = str(row["oclc_number"])
    print("Getting records for " + oclc_number + "\n")
    get_all_holdings_records(oclc_number)

Getting records for 824231

http://www.worldcat.org/webservices/catalog/content/libraries/824231?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/824231?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/824231?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/824231?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=301&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/824231?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZ

http://www.worldcat.org/webservices/catalog/content/libraries/632723490?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/632723490?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/632723490?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=301&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/632723490?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=401&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/632723490?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67

Getting records for 761985573

http://www.worldcat.org/webservices/catalog/content/libraries/761985573?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
Getting records for 778944557

http://www.worldcat.org/webservices/catalog/content/libraries/778944557?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
Getting records for 803708950

http://www.worldcat.org/webservices/catalog/content/libraries/803708950?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/803708950?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries

http://www.worldcat.org/webservices/catalog/content/libraries/949805301?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=601&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/949805301?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=701&maximumLibraries=100
Getting records for 953655841

http://www.worldcat.org/webservices/catalog/content/libraries/953655841?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/953655841?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
Getting records for 958105558

http://www.worldcat.org/webservices/catalog/content/libraries/958105558?location=m1b6j4&

http://www.worldcat.org/webservices/catalog/content/libraries/1074309573?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/1074309573?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/1074309573?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=301&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/1074309573?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=401&maximumLibraries=100
Getting records for 1074333271

http://www.worldcat.org/webservices/catalog/content/libraries/1074333271?location=m1b6j4&wskey=4udYRUkDSp7jMSJUS

...

http://www.worldcat.org/webservices/catalog/content/libraries/955920011?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=401&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/955920011?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=501&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/955920011?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=601&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/955920011?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=701&maximumLibraries=100
Getting records for 956629947

http://www.worldcat.org/webservices/catalog/content/libraries/956629947?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVm

http://www.worldcat.org/webservices/catalog/content/libraries/964866334?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/964866334?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
Getting records for 964923635

http://www.worldcat.org/webservices/catalog/content/libraries/964923635?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/964923635?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/964923635?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGy

http://www.worldcat.org/webservices/catalog/content/libraries/975090635?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/975090635?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/975090635?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=301&maximumLibraries=100
Getting records for 975369749

http://www.worldcat.org/webservices/catalog/content/libraries/975369749?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/975369749?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGy

http://www.worldcat.org/webservices/catalog/content/libraries/987867808?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/987867808?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/987867808?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=301&maximumLibraries=100
Getting records for 989024

http://www.worldcat.org/webservices/catalog/content/libraries/989024?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=1&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/989024?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8e

http://www.worldcat.org/webservices/catalog/content/libraries/999915482?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=101&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/999915482?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=201&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/999915482?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=301&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/999915482?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67GNKcOLini9gfP6PJuJOW8dkvLv&startLibrary=401&maximumLibraries=100
http://www.worldcat.org/webservices/catalog/content/libraries/999915482?location=m1b6j4&wskey=4udYRUkDSp7jMSJUSEvBDVmGyMaI8B3E8eLpbrby0dZNWaHPTZmq67